In [1]:
import numpy as np
import itertools, copy
import os

In [ ]:
# a function, generate all (possible) non-increasing outdegree sequences for simple graphs with p nodes
def OutdSeq(p, edges):
    seqlist = []
    for k in reversed(range(1,p)):
        stack = [(1, [k], k)]
        while stack:
            cur, seq, csum = stack.pop()
            if cur <= p:
                if csum == edges:
                    #seqlist.append(seq[:])
                    seqlist.append(seq + [0]*(p-cur))
                else:
                    for i in range(1,min(edges-csum, seq[-1], p-cur+1)+1):
                        csum += i
                        stack.append((cur+1, seq+[i], csum))
                        csum -= i
    return seqlist

In [ ]:
# a function, output all graphs for a specific outdegree sequence
def AllGraphs(seq):
    assert len(seq) == p
    
    # generate all graphs corresponding to a specific outdegree sequence, with tail nodes representation
    adj = np.array([[0 for _ in range(p)] for _ in range(p)])
    stack = [(0, [], adj)]
    tmp = []
    
    while stack:
        cur, path, adj = stack.pop() #matrix B, i,j entry is j->i
        if cur == p:
            tmp.append(path)
        else:
            if seq[cur] == 0:
                tmp.append(path + [set()] * (p-cur-1))
            else:
                for ch in list(itertools.combinations([x for x in range(p) if x != cur], seq[cur])): # all possible children set of size seq[cur]
                    ch = list(ch)
                    #print(cur, ch)
                    if sum(adj[ch, cur]) == 0 and sum(adj[cur, ch]) == 0: # no edge from ch(cur) to cur(ch), ch can be children set of cur
                        adj[ch, cur] = 1
                        stack.append((cur+1, path+[ch], copy.deepcopy(adj)))
                        adj[ch, cur] = 0 # *cur to children, reset to 0
                        
                    
    # transform to edge representation
    def trans(path):
        res = []
        for i, item in enumerate(path):
            if item:
                res.extend(list(map(lambda x: (i,x), item)))
        return res
    
    #return tmp
    return list(map(trans, tmp))

In [ ]:
# generate all simple graphs with p nodes, listed as a folder tree structure

p = 6
if not os.path.exists('p='+str(p)):
    os.mkdir('p='+str(p))

for edges in range(p, p*(p-1)//2):
    seqlist = OutdSeq(p, edges)
    os.mkdir('p='+str(p)+'//e='+str(edges))
    l = len(seqlist) // 10
    for idx, seq in enumerate(seqlist):
        with open('p=' + str(p) + '//e=' + str(edges) + '//'\
                  + str(idx).zfill(l), 'wt') as f:
            print(AllGraphs(seq), file=f)
            f.close()